In [53]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).cuda()

# Tokenize the sentence
sentence1 = "I want to be a banker."
sentence2 = "banker is great"
model
def vector(sentence1, sentence2):
    tokens1 = tokenizer.encode(sentence1, add_special_tokens=True)
    tokens2 = tokenizer.encode(sentence2, add_special_tokens=True)
    # Generate the sentence vector
    input_ids1 = torch.tensor([tokens1]).cuda()
    input_ids2 = torch.tensor([tokens2]).cuda()
    with torch.no_grad():
        output = model(input_ids1)
        vector1 = output.last_hidden_state.mean(dim=1).squeeze().cpu()
        output = model(input_ids2)
        vector2 = output.last_hidden_state.mean(dim=1).squeeze().cpu()
    return 1 - spatial.distance.cosine(vector1, vector2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [109]:
course_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Course_Name', 'Course_Description',
       'Course_Soft_Skills', 'Course_Hard_Skills'],
      dtype='object')

In [120]:
import pandas as pd
import numpy as np
job_df = pd.read_csv('job_with_skills.csv')
course_df = pd.read_csv('output_cleaned.csv')
course_df.dropna(subset=['Course_Hard_Skills'],inplace=True,axis=0)
course_df = course_df.reset_index(drop=True)
training_columns = ['Job','Softskills','Hardskills','Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills','label']
training_pd = pd.DataFrame(data=[[0, 1, 2, 3, 4, 5, 6, 7]], columns = training_columns)
for i in range(job_df.shape[0]):
    job_sen = job_df.loc[i,'Hardskills']
    scores = []
    for j in range(course_df.shape[0]):
        course_sen = course_df.loc[j,'Course_Hard_Skills']
        scores.append(vector(job_sen, course_sen))
    scores = np.array(scores)
    indexes = np.argsort(scores)[::-1]
    label_1 = indexes[:10]
    label_0 = np.random.choice(indexes[50:],20,replace = False)
    print(label_1,label_0)
    for idx in label_1:
        training_pd.loc[training_pd.shape[0],:]=list(job_df.loc[i,:])+list(course_df.loc[idx,['Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills']])+\
        [1]
    for idx in label_0:
        training_pd.loc[training_pd.shape[0],:]=list(job_df.loc[i,:])+list(course_df.loc[idx,['Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills']])+\
        [0]
training_pd = training_pd[1:].reset_index(drop=True)

[ 80 115  76 106  15 107 103  90 105 114] [  0  94 181 169 170  21  49 151  75 101  68  35 140 188 148  70 155 159
 189 162]
[ 76  80 107  72 152  74   9 106 105 127] [ 99  62  33  50  57  73 139   6 187 158 119  54 186  83   3  48  41  51
  89 143]
[ 76  80 105 107 115   9 116 106  58  15] [130 168 183 165   7  62 153  24  34  45 110 147 157 184  97  47  53  95
 178 125]
[ 76  80 107  72  74   9 106 152 124 105] [101 189 145 166 118  48 186  70 122  36  17  31  39  37   2  51 110 139
  89 146]
[115  15  76  86 124 107  77  91 106 167] [ 53  63 174  72  50   1 119  28 189  79  10 122  25 157  32   8  27  54
  31  37]
[ 76  80 107  72 106   9  74 152 124  26] [ 65 138  31 154 101  91  37  52 118 150  60  99 176 141  64 158  69  11
  36 180]
[ 76  80 107 106   9 115  74 152 124 127] [148  88 185 174  93 177  94  25  10  36  98   5 157 139  50  66 162 109
 145  60]
[ 76 115 107 106   9 124  80 164 167 105] [133  55 130 125  84 191  67  16 184  53 188  66  36 141 120 180 169  93
  50  40]


In [121]:
training_pd

,Job,Softskills,Hardskills,Course_Name,Course_Description,Course_Hard_Skills,Course_Soft_Skills,label
0,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Case studies in corporate finance,structured around the most important financial...,"financial analysis, decision making, risk asse...",NaN,1.0
1,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Behavioural and sociological finance,covers current developments in finance. Possib...,"Understanding current developments in finance,...",NaN,1.0
2,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Senior seminar in economics and finance,applies core theoretical knowledge in economic...,"research skills, theoretical knowledge in econ...",NaN,1.0
3,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Current issues in asset management and private...,seeks to cover current development and current...,"Asset management, Private banking, Identifying...",NaN,1.0
4,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Valuation using financial statements,o introduce students to the concepts of financ...,Basic skills and techniques to analyze financi...,NaN,1.0
...,...,...,...,...,...,...,...,...
565,Industry Analyst,"Analytical skills, problem-solving, attention ...","Industry analysis, data analysis, IT skills, k...",e-Commerce,focuses on business opportunities brought abou...,"internet commerce, web development, and busine...",NaN,0.0
566,Industry Analyst,"Analytical skills, problem-solving, attention ...","Industry analysis, data analysis, IT skills, k...",International business environment,helps you to understand the macroeconomic envi...,"understanding macroeconomic environments, econ...",NaN,0.0
567,Industry Analyst,"Analytical skills, problem-solving, attention ...","Industry analysis, data analysis, IT skills, k...",Chinese economy,provides a general introduction of Chinese eco...,knowledge of Chinese economic history since 19...,ability to understand the development and tran...,0.0
568,Industry Analyst,"Analytical skills, problem-solving, attention ...","Industry analysis, data analysis, IT skills, k...",Lending and credit in private banking and corp...,discuss the bank’s credit process including pu...,"Understanding of the bank's credit process, in...",NaN,0.0


In [122]:
training_pd.to_csv('training_pd.csv',index=False)